In [1]:
import os
import cv2
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pickle

from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt
from torchsummary import summary

In [2]:
class FCNN(nn.Module):
    """
    This is a class for fully convolutional neural networks.
    
    It is a subclass of the Module class from torch.nn.
    See the torch.nn documentation for more information.
    """
    
    def __init__(self):
        """
        The constructor for FCNN class. The internal states of the network are initialized. 
        """
        
        super(FCNN, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=1, out_channels=30, kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=30, out_channels=30, kernel_size=5, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=30, out_channels=60, kernel_size=3, stride=1)
        self.conv2_drop = nn.Dropout2d()
        
        self.conv3 = nn.Conv2d(in_channels=60, out_channels=60, kernel_size=3, stride=2)
        self.conv3_drop = nn.Dropout2d()
        
        self.conv4 = nn.Conv2d(in_channels=60, out_channels=120, kernel_size=3, stride=1)
        
        self.conv5 = nn.Conv2d(in_channels=120, out_channels=120, kernel_size=3, stride=1)
        
        self.conv6 = nn.Conv2d(in_channels=120, out_channels=1, kernel_size=1, stride=1)
        
        
    def forward(self, data):
        """
        Defines the computation performed at every call.
        
        Parameters:
            data (torch.Tensor): The input that is evaluated by the network. 
                The network expects the input to be of 4 dimensions.
            
        Returns:
            x (torch.Tensor): The output of the network after evaluating it on the given input.
        """
        
        x = F.relu(self.conv0(data))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2_drop(self.conv2(x)))
        x = F.relu(self.conv3_drop(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = torch.sigmoid(self.conv6(x))
        return x

# Evaluate Network

### Load Network

In [10]:
# define which network to load
cur_dir = os.getcwd()
path_to_network = os.path.join(cur_dir, 'pollennet.pt')

# initialise blank network
pollen_network = FCNN()
# update with saved weights

pollen_network.load_state_dict(torch.load(path_to_network, map_location={'cuda:0': 'cpu'}))
pollen_network.eval()
# define criterion and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(pollen_network.parameters(), lr=0.001, momentum=0.9)

# # load validloader and losses:
# TODO
# printloader should load validloader with torch.load() somehow
# losses with pickle?

### Show loaded weights

In [11]:
print("\n-----------------------------\n")
print("Weights of the first layer:")
print(pollen_network.conv0.weight)

print("\n-----------------------------\n")
print("Model's state_dict:")
for param_tensor in pollen_network.state_dict():
    print(param_tensor, "\t", pollen_network.state_dict()[param_tensor].size())

print("\n-----------------------------\n")
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])


-----------------------------

Weights of the first layer:
Parameter containing:
tensor([[[[ 0.0986, -0.0424],
          [ 0.2190,  0.2284]]],


        [[[ 0.1057, -0.0636],
          [ 0.3646,  0.2155]]],


        [[[ 0.0266, -0.2808],
          [ 0.1276, -0.1018]]],


        [[[-0.2647, -0.0161],
          [-0.2710, -0.4592]]],


        [[[-0.4651, -0.1534],
          [ 0.3088,  0.1992]]],


        [[[-0.2286, -0.1250],
          [-0.3302, -0.3245]]],


        [[[ 0.2022,  0.4456],
          [ 0.3600,  0.4520]]],


        [[[-0.0997, -0.3818],
          [ 0.1190, -0.1973]]],


        [[[-0.3771,  0.4947],
          [ 0.0497,  0.3222]]],


        [[[-0.2047, -0.1365],
          [-0.4003, -0.0844]]],


        [[[ 0.2733,  0.2067],
          [-0.2521,  0.0673]]],


        [[[-0.0484, -0.4543],
          [ 0.3958,  0.0291]]],


        [[[-0.4005,  0.4118],
          [-0.1296, -0.3436]]],


        [[[ 0.2999,  0.0191],
          [ 0.2623, -0.3160]]],


        [[[-0.0740,  0

### Show how data is transformed in size while going through the network

In [ ]:
summary(pollen_network, (1,64,64))
print("-----------------------------")
summary(pollen_network, (1,4000,3000))

### Plot Losses

In [12]:
plt.plot(train_losses)
plt.plot(val_losses)
plt.plot(F1_scores)
plt.plot(accuracies)
plt.legend(["Train Loss", "Validation Loss", "F1-Score", "Accuracy"])
plt.show()

NameError: name 'train_losses' is not defined

### Plot some wrong classified samples

In [13]:
# This function was used to find the weaknesses of the network.

def print_false_positive(network, testloader):
    """
    Evaluates the network and prints the false positives samples. 
    
    Parameters:
    
    Returns:
    
    """
    
    counter = 0
    # We do not need any gradiants here, since we do not train the network.
    # We are only interested in the predictions of the network on the testdata. 
    with torch.no_grad():
        for i, sample in enumerate(testloader):
#         for i, (inputs, labels) in enumerate(testloader):
            inputs = sample['image']
            labels = sample['label']
            outputs = network(torch.transpose(inputs[...,None],1,3)).view(-1)
            predicted = (outputs >= threshold) # Predicted is a tensor of booleans 
            predicted = predicted.view(predicted.size(0))
            labels = labels != 0
            if (predicted and not labels):
                title = 'Label: False, Predicted: True'
                fig, ax = plt.subplots()
                plt.imshow(np.array(inputs[0]), cmap='gray')
                ax.set_title(title)
                plt.show()
                counter += 1
                if counter == 20:
                    break
    return counter

def print_false_negative(network, testloader):
    # We do not need any gradiants here, since we do not train the network.
    # We are only interested in the predictions of the network on the testdata. 
    counter = 0
    with torch.no_grad():
        for i, sample in enumerate(testloader):
#         for i, (inputs, labels) in enumerate(testloader):

            inputs = sample['image']
            labels = sample['label']
            outputs = network(torch.transpose(inputs[...,None],1,3)).view(-1)
            predicted = (outputs >= threshold) # Predicted is a tensor of booleans 
            predicted = predicted.view(predicted.size(0))
            labels = labels != 0
            if (not predicted and labels):
                title = 'Label: True, Predicted: False'
                fig, ax = plt.subplots()
                plt.imshow(np.array(inputs[0]), cmap='gray')
                ax.set_title(title)
                plt.show()
                counter += 1
                if counter == 20:
                    break
    return counter

In [15]:
print_false_positive(pollen_network, printloader)
print("-----------------------------")
print_false_negative(pollen_network, printloader)

NameError: name 'printloader' is not defined

# Create Heatmaps

### Load Data

In [ ]:
def get_samples(dir):
    """returns samples in given directory"""

    # samples will be a list of tuples, each tuple contains a path and a list of coords of a single image
    samples = []
    for root, folders, files in os.walk(dir):
        for folder in folders:
            if folder == 'img':
                for img_root, img_folder, img_files in os.walk(os.path.join(root, folder)):
                    for img_file in img_files:
                        # go find its related annotation file:
                        found_ann = False
                        for ann_root, ann_folder, ann_files in os.walk(os.path.join(root, 'ann')):
                            if found_ann:
                                break
                            for ann_file in ann_files:
                                if img_file in ann_file:
                                    # found a pair!
                                    cur_coords = []
                                    with open(os.path.join(ann_root, ann_file)) as ann_json:
                                        ann_data = json.load(ann_json)
                                    cur_len = len(ann_data['objects'])
                                    if cur_len:
                                        for obj in ann_data['objects']:
                                            cur_coords.append(obj['points']['exterior'][0])
                                        img_path = os.path.join(img_root, img_file)
                                        np_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                                        samples.append((np.array(np_img), cur_coords))
                                    # to prevent unnecessary looping:
                                    found_ann = True
                                    break
    return samples

### Create Heatmaps

In [ ]:
def heat(network, samples):
    """
    Feeds images of any size into the network and returns the belonging heatmaps. 
    
    Parameters:
        network (FCNN): A fully convolutional neural network, that is used to compute a heatmap.
        testloader (torch.utils.data.DataLoader): Dataloader with batch size 1 that contains the
            images you want to compute the heatmaps of
        
    Returns:
        heatmaps ([np.array]): A list which contains a heatmap that is a 2D array for every input image.
    """
    
    heatmaps = []
    print('Creating heatmaps...')
    with torch.no_grad():

        for fs_image, fs_coords in samples:
            t_image = torch.Tensor(fs_image)[None, ...]
            # outputs = network(torch.transpose(inputs[...,None],1,3))
            heatmap = network(torch.transpose(t_image[...,None],1,3))
            heatmaps.append((np.array(torch.transpose(heatmap[0,0,:,:],0,1)),fs_coords))
            
    print('Heatmaps created.')
    return heatmaps

### Find local maxima with max-pooling

In [ ]:
def non_max_suppression(heatmaps, local_size = 8):
    heatmaps_sup = []
    pooling = nn.MaxPool2d((local_size * 2 - 1), stride = 1, padding = local_size - 1)
    for heatmap in heatmaps:
        max_filter = pooling(torch.tensor(heatmap)[None,...])
        max_filter = np.array(max_filter)
        heatmap = ((heatmap == max_filter) * (heatmap >= 0.8)).astype(int)
        heatmaps_sup.append(heatmap[0,:,:])
        
    return heatmaps_sup

def non_max_suppression_single(heatmap, local_size = 8):
    pooling = nn.MaxPool2d((local_size * 2 - 1), stride = 1, padding = local_size - 1)
    max_filter = pooling(torch.tensor(heatmap)[None,...])
    max_filter = np.array(max_filter)
    heatmap = ((heatmap == max_filter) * (heatmap >= 0.8)).astype(int)
    heatmap_sup = heatmap[0,:,:]
        
    return heatmap_sup

### Extract coordiantes of predicted pollen in heatmap

In [ ]:
def pollen_coordinates(heatmap):
    coordinate_list = []
    h_coordinates = np.argwhere(heatmap == 1)
    for i in range(h_coordinates.shape[0]):
        coordinate_list.append((h_coordinates[i,0] * 2 * 2 * 2 + 28, h_coordinates[i,1] * 2 * 2 * 2 + 28))
    return coordinate_list

### Plot Image with actual Pollen and predicted locations

In [ ]:
def guess_plotter(img, network_guess, actual_points):

    np_img = np.array(Image.open(img), dtype=np.uint8)
    main_fig,ax = plt.subplots(1)
    ax.imshow(np_img)

    for coord in actual_points:
        crcl = patches.Circle((coord[0],coord[1]),35,linewidth=4,edgecolor='r',facecolor='none')
        ax.add_patch(crcl)

    for coord in network_guess:
        crcl = patches.Circle((coord[0],coord[1]),20,linewidth=2,edgecolor='b',facecolor='none')
        ax.add_patch(crcl)

    plt.show()

### Run the code above:

In [ ]:
# Load Images
cur_dir = os.getcwd()
full_size_path = os.path.join(curd_dir, 'Fullsize')
full_size_images = get_samples(full_size_path)

# Create heatmap of images
heatmaps = heat(network,full_size_images)

# Find local maxima on heatmaps
non_max_heatmaps = [(non_max_suppression_single(heatmap),coords) for heatmap, coords in heatmaps]

# TODO
# plot images with predicted and real pollen marked